In [1]:
import sys
import os
import gc
import cv2
import math
import time
import tqdm
import random
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import KFold,StratifiedKFold

import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
from torch.nn import Parameter
import torch.nn.functional as F
from torch.optim import Adam, lr_scheduler, AdamW
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import _LRScheduler
from torch.optim.lr_scheduler import (CosineAnnealingWarmRestarts, CosineAnnealingLR, 
                                      ReduceLROnPlateau)

from sklearn.preprocessing import LabelEncoder

import albumentations as A 
from albumentations.pytorch.transforms import ToTensorV2

import transformers
from transformers import AutoModel, AutoTokenizer

from colorama import Fore, Back, Style
y_ = Fore.YELLOW
r_ = Fore.RED
g_ = Fore.GREEN
b_ = Fore.BLUE
m_ = Fore.MAGENTA
c_ = Fore.CYAN
sr_ = Style.RESET_ALL

In [2]:
train_path = '../input/shopee-product-matching/train_images/'
test_path = '../input/shopee-product-matching/test_images/'

train_data = pd.read_csv('../input/shopee-product-matching/train.csv').head(2000)
test_data = pd.read_csv('../input/shopee-product-matching/test.csv')

train_data['image_paths'] = train_path + train_data['image']
test_data['image_paths'] = test_path + test_data['image']

le = LabelEncoder()
train_data['label'] = le.fit_transform(train_data['label_group'])

In [3]:
config = {
    'learning_rate':0.01,
    'train_batch_size':12,
    'valid_batch_size':12,
    'accumulation_step':4,
    'epochs':15,
    'nfolds':5,
    'seed':42,
    
    's':30.0,
    'm':0.5,
    'ls_eps':0.0,
    'easy_margin':False,
}

In [4]:
def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONASSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(seed=config['seed'])

In [5]:
 class ShopeeDataset(nn.Module):
    def __init__(self,df,tokenizer):
        self.titles = df['title'].to_numpy()
        self.labels = df['label'].to_numpy()
        self.tokenizer = tokenizer
    
    def __getitem__(self,idx):
        label = torch.as_tensor(self.labels[idx],dtype=torch.long)
        encode = self.tokenizer(self.titles[idx],return_tensors='pt',padding='max_length',truncation=True)
        return encode,label
    
    def __len__(self):
        return len(self.titles)

In [6]:
class ArcMarginProduct(nn.Module):
    def __init__(self, in_features, out_features, s=30.0, m=0.50, easy_margin=False, ls_eps=0.0):
        super(ArcMarginProduct, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.s = s
        self.m = m
        self.ls_eps = ls_eps  
        self.weight = Parameter(torch.FloatTensor(out_features, in_features))
        nn.init.xavier_uniform_(self.weight)

        self.easy_margin = easy_margin
        self.cos_m = math.cos(m)
        self.sin_m = math.sin(m)
        self.th = math.cos(math.pi - m)
        self.mm = math.sin(math.pi - m) * m

    def forward(self, input, label):
        cosine = F.linear(F.normalize(input), F.normalize(self.weight)).to('cuda')
        sine = torch.sqrt(1.0 - torch.pow(cosine, 2)).to('cuda')
        phi = (cosine * self.cos_m - sine * self.sin_m).to('cuda')
        if self.easy_margin:
            phi = torch.where(cosine > 0, phi, cosine)
        else:
            phi = torch.where(cosine > self.th, phi, cosine - self.mm)
        one_hot = torch.zeros(cosine.size(), device='cuda')
        one_hot.scatter_(1, label.view(-1, 1).long(), 1)
        if self.ls_eps > 0:
            one_hot = (1 - self.ls_eps) * one_hot + self.ls_eps / self.out_features
        output = (one_hot * phi) + ((1.0 - one_hot) * cosine)
        output *= self.s

        return output

In [7]:
class Model(nn.Module):
    def __init__(self,backbone):
        super(Model,self).__init__()
        self.backbone = backbone
        self.in_features = self.backbone.pooler.dense.in_features
        self.dropout = nn.Dropout(0.2)
        self.final = ArcMarginProduct(self.in_features, 11014,
                                      s=config['s'], m=config['m'],
                                      easy_margin=config['easy_margin'], 
                                      ls_eps=config['ls_eps'])
    
    def forward(self,x,label):
        output = self.backbone(**x)
        x = output['last_hidden_state'][:,0,:]
        x = self.dropout(x)
        x = self.final(x,label)
        return x  

In [8]:
MODEL_PATH = 'bert-base-cased'
# backbone = AutoModel.from_pretrained(MODEL_PATH)
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)
tokenizer.save_pretrained('bert_tokenizer')

# model.save_pretrained('bert_model')
backbone = AutoModel.from_pretrained("bert-base-cased")
bert_config = backbone.config
bert_config.add_cross_attention = True
bert_config.is_decoder = True
backbone = AutoModel.from_pretrained("bert-base-cased", config=bert_config)
backbone

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/213k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/436k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of BertModel were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['bert.encoder.layer.0.crossattention.self.query.weight', 'bert.encoder.layer.0.crossattention.self.query.bias', 'bert.encoder.layer.0.crossattention.self.key.weight', 'bert.encoder.layer.0.crossattention.self.key.bias', 'bert.encoder.layer.0.crossattention.self.value.weight', 'bert.encoder.layer.0.crossattention.self.value.bias', 'bert.encoder.layer.0.crossattention.output.dense.weight', 'bert.encoder.layer.0.crossattention.output.dense.bias', 'bert.encoder.layer.0.crossattention.output.LayerNorm.weight', 'bert.encoder.layer.0.crossattention.output.LayerNorm.bias', 'bert.encoder.layer.1.crossattention.self.query.weight', 'bert.encoder.layer.1.crossattention.self.query.bias', 'bert.encoder.layer.1.crossattention.self.key.weight', 'bert.encoder.layer.1.crossattention.self.key.bias', 'bert.encoder.layer.1.crossattention.self.value.weight', 'bert.encoder.layer.1.crossatte

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(28996, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          

In [9]:
def run(plot_losses=True, verbose=True):
        
    def loss_fn(inputs,targets):
        return nn.CrossEntropyLoss()(inputs,targets)
    
    def train_loop(train_loader, model, loss_fn, device,optimizer,lr_scheduler=None):
        model.train()
        total_loss = 0
        model.zero_grad(set_to_none=True)
        for i, (inputs,targets) in enumerate(train_loader):
            inputs = {key:val.reshape(val.shape[0],-1).to(device) for key,val in inputs.items()}
            targets = targets.to(device)
            outputs = model(inputs,targets)
            loss = loss_fn(outputs,targets) / config['accumulation_step']
            loss.backward()
            if (i + 1) % config['accumulation_step'] == 0:
                optimizer.step()
                model.zero_grad(set_to_none=True)
                
            total_loss += loss.item()
        total_loss /= len(train_loader)
        return total_loss
    
    def valid_loop(valid_loader, model, loss_fn, device):
        model.eval()
        total_loss = 0
        with torch.no_grad():
            for i, (inputs,targets) in enumerate(valid_loader):
                inputs = {key:val.reshape(val.shape[0],-1).to(device) for key,val in inputs.items()}
                targets = targets.to(device)
                outputs = model(inputs,targets)
                loss = loss_fn(outputs,targets)
                total_loss += loss.item()
            total_loss /= len(valid_loader)
        return total_loss 
    
    fold_train_losses = list()
    fold_valid_losses = list()
    
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"{device} is used in this run")
    
    train = train_data
    kfold = StratifiedKFold(n_splits=config['nfolds'], shuffle=True, random_state=0)
    for k , (train_idx,valid_idx) in enumerate(kfold.split(X=train,y=train['label'])):
        x_train,x_valid = train.loc[train_idx],train.loc[valid_idx]

        train_ds = ShopeeDataset(x_train,tokenizer)
        train_dl = DataLoader(train_ds,
                              batch_size = config["train_batch_size"],
                              shuffle=True,
                              num_workers = 4,
                              pin_memory=True
                             )

        valid_ds = ShopeeDataset(x_valid,tokenizer)
        valid_dl = DataLoader(valid_ds,
                              batch_size = config["valid_batch_size"],
                              shuffle=False,
                              num_workers = 4,
                              pin_memory=True,
                              drop_last=False,
                             )
        
        model = Model(backbone)
        model.to(device)
        
        optimizer = optim.AdamW(model.parameters(),lr=config['learning_rate'])
        lr_scheduler = None
    

        print(f"Fold {k}")
        best_loss = 99999
        
        train_losses = list()
        valid_losses = list()
        start = time.time()
        for i in range(config["epochs"]):
            train_loss = train_loop(train_dl,model,loss_fn,device,optimizer,lr_scheduler=lr_scheduler)
            valid_loss = valid_loop(valid_dl,model,loss_fn,device)
            
            if lr_scheduler:
                lr_scheduler.step(valid_loss)

            train_losses.append(train_loss)
            valid_losses.append(valid_loss)
            
            end = time.time()
            epoch_time = end - start
            start = end
                                      
            if verbose:
                print(f"epoch:{i} Training loss:{train_loss} | Validation loss:{valid_loss}| epoch time {epoch_time:.2f}s ")

            if valid_loss <= best_loss:
                if verbose:
                    print(f"{g_}Validation loss Decreased from {best_loss} to {valid_loss}{sr_}")
                best_loss = valid_loss
                torch.save(model.state_dict(),f'bert.bin')
                
        fold_train_losses.append(train_losses)
        fold_valid_losses.append(valid_losses)
        
    if plot_losses == True:
        plt.figure(figsize=(20,14))
        for i, (t,v) in enumerate(zip(fold_train_losses,fold_valid_losses)):
            plt.subplot(2,5,i+1)
            plt.title(f"Fold {i}")
            plt.plot(t,label="train_loss")
            plt.plot(v,label="valid_loss")
            plt.legend()
        plt.show()

In [ ]:
run()

cuda is used in this run
Fold 0
epoch:0 Training loss:6.3908182293621465 | Validation loss:25.873302684110755| epoch time 172.90s 
Validation loss Decreased from 99999 to 25.873302684110755
epoch:1 Training loss:6.480274203997939 | Validation loss:26.180844699635227| epoch time 176.27s 
epoch:2 Training loss:5.831693884152085 | Validation loss:24.434434666353113| epoch time 174.34s 
Validation loss Decreased from 25.873302684110755 to 24.434434666353113
